In [ ]:
# Function to initialize widgets for user input
def create_widgets():
    dbutils.widgets.text("file_location", "s3/file/path", "Upload Location")
    dbutils.widgets.dropdown("file_type", "json", ["csv", 'parquet', 'json'])

# Function to read data from S3 and return a DataFrame
def read_data_from_s3(file_type, file_location):
    return spark.read.format(file_type) \
                     .option("inferSchema", "true") \
                     .load(file_location).limit(1)

# Function to fetch Snowflake credentials and prepare options
def get_snowflake_options():
    user = dbutils.secrets.get("snowflake_credentials", "username")
    password = dbutils.secrets.get("snowflake_credentials", "password")
    return {
        "sfUrl": "url",
        "sfUser": user,
        "sfPassword": password,
        "sfDatabase": "database",
        "sfSchema": "schema",
        "sfWarehouse": "warehouse"
    }

# Function to write a DataFrame to a Snowflake table
def write_to_snowflake(df, options, table_name="table"):
    df.write \
      .format("snowflake") \
      .options(**options) \
      .option("dbtable", table_name) \
      .mode("append") \
      .save()

# Main function to run the workflow
def main():
    try:
        create_widgets()
        
        file_type = dbutils.widgets.get("file_type")
        file_location = dbutils.widgets.get("file_location")
        
        df = read_data_from_s3(file_type, file_location)
        df.show()

        snowflake_options = get_snowflake_options()
        write_to_snowflake(df, snowflake_options)
    except Exception as e:
        print(f"An error occurred: {e}")

# main()
